In [ ]:
import pysmile
import pysmile_license
import numpy as np
import pandas as pd

from df_plot import plot_df 
np.seterr(divide='ignore', invalid = 'ignore')

In [ ]:
# Read the network
net = pysmile.Network()
net.read_file("genie_models/Basic_ID_screening_06032024.xdsl")

net.set_evidence("CRC", False)
p_CRC_false = net.prob_evidence()

net.set_evidence("CRC", True)
p_CRC_true = net.prob_evidence()

### Cost efectiveness

In [ ]:
net.get_all_node_ids()

['Sex',
 'Age',
 'SD',
 'SES',
 'Depression',
 'PA',
 'Smoking',
 'Anxiety',
 'Alcohol',
 'BMI',
 'Hyperchol_',
 'Diabetes',
 'Hypertension',
 'CRC',
 'Screening',
 'Results_of_Screening',
 'Colonoscopy',
 'Results_of_Colonoscopy',
 'Value_of_CRC_detection',
 'Cost_of_Screening',
 'Patient_comfort',
 'Complications',
 'Cost_of_complications',
 'Utility']

In [ ]:
net.update_beliefs()

In [ ]:
net.get_mau_expressions("Utility")

['Cost_of_Screening+Cost_of_complications+100*Patient_comfort+3e+06*Value_of_CRC_detection']

In [ ]:
wtp = 10
expression = f'1*Cost_of_Screening+{wtp}*Value_of_CRC_detection+Cost_of_complications+Patient_comfort'

In [ ]:
net.set_mau_expressions(node_id = "Utility", expressions = [expression])

In [ ]:
net.get_node_value("Utility")

SMILEException: SMILE Error 1 in function Network.GetValue

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import math

class Inf_diag_inference:
    def __init__(self):
        print("Starting inference...")
        inf_diag = pysmile.Network()
        inf_diag.read_file("genie_models/Basic_ID_screening.xdsl")
        
        #print("No evidence set.")
        #inf_diag.update_beliefs()
        #self.print_cost_utility(inf_diag)

        node_id = ["Sex", "Age", "PA", "Smoking", "Alcohol", "BMI"]
        outcome_id = ["W", "age_5_old_adult", "PA_2", "sm_3_ex_smoker", "low", "bmi_4_obese"]
        self.change_evidence_and_update(inf_diag, node_id, outcome_id)

        tot_array = self.plot_wtp(inf_diag)

        for tool in range(tot_array.shape[1]):
            plt.plot(tot_array[:,tool])

        '''print("Setting Forecast=Good.")
        self.change_evidence_and_update(net, "Forecast", "Good")
        print("Adding Economy=Up")
        self.change_evidence_and_update(net, "Economy", "Up")'''

        print("Inference complete.")

    def change_evidence_and_update(self, net, node_id, outcome_id):
        for i in range(len(node_id)):
            if outcome_id[i] is not None:
                net.set_evidence(node_id[i], outcome_id[i])
            else:
                net.clear_evidence(node_id[i])

        net.update_beliefs()
        self.print_cost_utility(net)

    def print_cost_utility(self, net):
        expected_utility = net.get_node_value("Cost_utility")
        util_parents = net.get_value_indexing_parents("Cost_utility")
        self.print_utility_matrix(net, expected_utility, util_parents)

    def print_utility_matrix(self, net, mtx, parents):
        dim_count = 1 + len(parents)
        dim_sizes = [0] * dim_count
        for i in range(0, dim_count - 1):
            dim_sizes[i] = net.get_outcome_count(parents[i])
        dim_sizes[len(dim_sizes) - 1] = 1
        coords = [0] * dim_count
        for elem_idx in range(0, len(mtx)):
            self.index_to_coords(elem_idx, dim_sizes, coords)
            str_to_print = " Utility("
            if dim_count > 1:
                for parent_idx in range(0, len(parents)):
                    if parent_idx > 0:
                        str_to_print += ","
                    parent_handle = parents[parent_idx]
                    str_to_print += net.get_node_id(parent_handle) + \
                    "=" + net.get_outcome_id(parent_handle,
                                                coords[parent_idx])
            str_to_print += ")=" + str(mtx[elem_idx])
            print(str_to_print)
        print("")

    def index_to_coords(self, index, dim_sizes, coords):
        prod = 1
        for i in range(len(dim_sizes) - 1, -1, -1):
            coords[i] = int(index / prod) % dim_sizes[i]
            prod *= dim_sizes[i]

    def plot_wtp(self, net ):
        total_array = []
        for wtp in range(0, 20, 1):
            print(np.log(wtp))
            expression = f'1*Cost_of_Screening+{wtp}*Utility_of_Screening+Cost_of_Perforation+Cost_of_Bleeding+Cost_of_other_complications'
            net.set_mau_expressions(node_id = "Cost_utility", expressions = [expression])
            net.update_beliefs()
            total_array.append(net.get_node_value("Cost_utility"))

        np_tot_array = np.array(total_array)
        np_tot_array.reshape(-1,7)
        print(np_tot_array)
        return np_tot_array


In [ ]:
Inf_diag_inference()

Starting inference...


SMILEException: SMILE Error Occured. Invalid node handle: -2